### 
Stadium - 'MA Chidambaram Stadium, Chepauk'\
Phase - Middle Overs \
Opposition Team - Chennai Super Kings


In [248]:
import math
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 0)

import matplotlib.pyplot as plt
deliveries =pd.read_csv(r'C:\Users\kushs\Downloads\Datasets\deliveries.csv')
matches = pd.read_csv(r"C:\Users\kushs\Downloads\matches.csv")

In [249]:
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1
    
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [259]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    
    df=df[df.venue== current_venue]
    df=df[df.phase== current_phase]
    df=df[df.bowling_team	== current_opposition]
    df.reset_index(inplace = True , drop = True)
    
    df['isDot']= df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne']= df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo']= df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree']= df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour']= df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix']= df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)
    
    runs=pd.DataFrame(df.groupby([ 'batsman'])['batsman_runs'].sum()).rename(columns={'batsman_runs':'runs'})
    balls= pd.DataFrame(df.groupby([ 'batsman'])['match_id'].count()).rename(columns={'match_id':'balls'})
    innings=pd.DataFrame(df.groupby([ 'batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns= {'match_id':'innings'})
    dismissals=pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissal'})
    sixes=df.groupby(['batsman'])['isSix'].sum().reset_index().rename(columns= {'isSix': 'sixes'})
    fours=df.groupby(['batsman'])['isFour'].sum().reset_index().rename(columns= {'isFour': 'fours'})
    dots=df.groupby(['batsman'])['isDot'].sum().reset_index().rename(columns= {'isDot': 'dots'})

    df=pd.merge(innings, runs, on ='batsman').merge(balls, on = 'batsman').merge(dismissals, on = 'batsman').merge(fours, on = 'batsman').merge(sixes, on = 'batsman').merge(dots, on = 'batsman')
    df['RPI']= df.apply(lambda x: (x['runs']/x['innings']), axis=1)
    df['SR']= df.apply(lambda x: 100*(x['runs']/x['balls']), axis=1)
    df['BPB']= df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis=1)
    df['dot%']= df.apply(lambda x: (x['dots']/ x['balls']), axis=1)
    
    return df

In [260]:
df= deliveries.copy()

In [261]:
mdf=matches.copy()

In [262]:

mdf.rename(columns={'id':'match_id'},inplace=True)

In [263]:
comb= pd.merge(df, mdf, on='match_id' ,how='left')

In [264]:
comb['over_no']= comb['over'].apply(lambda x: x-1)

In [265]:
comb['phase']= comb['over_no'].apply(lambda x: current_phase(x))

In [266]:
df = ByCustom(comb, 'MA Chidambaram Stadium, Chepauk' , 'Middle', 'Chennai Super Kings')

In [281]:
df.sort_values(by ='runs', ascending= False).head(20).reset_index(drop=True)

,batsman,innings,runs,balls,dismissal,fours,sixes,dots,RPI,SR,BPB,dot%
0,V Kohli,7,175,150,4,11,5,50,25.000000,116.666667,9.375000,0.333333
1,JH Kallis,5,124,124,3,10,1,39,24.800000,100.000000,11.272727,0.314516
2,SR Watson,4,121,70,2,9,7,16,30.250000,172.857143,4.375000,0.228571
3,SE Marsh,3,104,54,1,8,7,10,34.666667,192.592593,3.600000,0.185185
4,MS Bisla,3,89,66,2,6,4,20,29.666667,134.848485,6.600000,0.303030
5,AB de Villiers,4,88,61,1,10,1,17,22.000000,144.262295,5.545455,0.278689
6,G Gambhir,3,86,73,1,4,2,21,28.666667,117.808219,12.166667,0.287671
7,R Dravid,4,79,74,2,9,0,27,19.750000,106.756757,8.222222,0.364865
8,DA Warner,3,72,73,2,1,3,28,24.000000,98.630137,18.250000,0.383562
9,Y Venugopal Rao,3,67,51,1,2,5,16,22.333333,131.372549,7.285714,0.313725
